In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
import re
import matplotlib.pyplot as plt

from imblearn.under_sampling import RandomUnderSampler
from sklearn.datasets import make_classification

%matplotlib inline
# ignore warnings
import warnings
warnings.filterwarnings("ignore")


In [2]:
df = pd.read_json('./data/dataSet_Culture_06102023-POINT.json')

In [3]:
# Extract the year from 'Analysis Date'
df['year'] = df['Analysis Date'].apply(lambda x: x.split('-')[0])

df['polygon_x'] = df['polygon'].apply(lambda x: x['x'])
df['polygon_y'] = df['polygon'].apply(lambda x: x['y'])
df['month'] = df['Analysis Date'].apply(lambda x: x.split('-')[1])
df['day'] = df['Analysis Date'].apply(lambda x: x.split('-')[2])

df['vegetation'] = (df['indextype'] == 'NDVI') & (df['averagevalue'] >= 0.15)
df = df.drop(['polygon', 'soil_id'] , axis = 1)

# Modify the 'combined' column to include year
df['combined'] = df['polygon_x'].astype(str) + '_' + df['polygon_y'].astype(str) + '_' + df['year'].astype(str)

# Assign unique ID based on the grouped column
df['id'] = df.groupby('combined').ngroup() + 1

# Drop the combined column and other temporary columns
df = df.drop(columns=['combined', 'polygon_x', 'polygon_y', 'year'])

In [4]:
df = df[df['indextype'] == 'NDVI']
df = df.drop(columns=['indextype', 'year contour', 'month', 'day', 'vegetation', 'type_culture_name'])
df = df.rename(columns={'culture_name': 'class'})
df = df.rename(columns={'averagevalue': 'red'})
df = df.rename(columns={'Analysis Date': 'date'})
df['date'] = pd.to_datetime(df['date'])

In [5]:
label_encoder = LabelEncoder() 
print(df['class'].unique() )
# Encode labels in column 'class'. 
df['district_name']= label_encoder.fit_transform(df['district_name']) 
df['soil_name']= label_encoder.fit_transform(df['soil_name']) 
df['class'].unique() 

['Картофель' 'Соя' 'Ячмень' 'Кукуруза' 'Пшеница' 'Свекла' 'Люцерна'
 'Яровой' 'Фасоль' 'Клевер' 'Арбуз' 'Лук' 'Томат' 'Морковь' 'Рапс'
 'Помидор' 'Софлор' 'Сил' 'Лук-Морковка' 'Капуста' 'Экпарцет'
 'Подсолнечник']


array(['Картофель', 'Соя', 'Ячмень', 'Кукуруза', 'Пшеница', 'Свекла',
       'Люцерна', 'Яровой', 'Фасоль', 'Клевер', 'Арбуз', 'Лук', 'Томат',
       'Морковь', 'Рапс', 'Помидор', 'Софлор', 'Сил', 'Лук-Морковка',
       'Капуста', 'Экпарцет', 'Подсолнечник'], dtype=object)

In [6]:
# Step 1: Create a mapping dictionary
group_mapping = {
    'Ячмень': 'Cereals', 'Пшеница': 'Cereals', 'Кукуруза': 'Cereals', 'Яровой': 'Cereals',
    'Соя': 'Legumes', 'Фасоль': 'Legumes', 'Клевер': 'Legumes', 'Люцерна': 'Legumes',
    'Картофель': 'Root Crops', 'Свекла': 'Root Crops', 'Морковь': 'Root Crops', 'Лук-Морковка': 'Root Crops',
    'Рапс': 'Oilseeds', 'Подсолнечник': 'Oilseeds', 'Софлор': 'Oilseeds',
    'Лук': 'Vegetables', 'Томат': 'Vegetables', 'Помидор': 'Vegetables', 'Капуста': 'Vegetables',
    'Арбуз': 'Melons',
    # Add more mappings as needed
}

# Step 2: Function to map class to group
def map_class_to_group(class_name):
    # Handle cases where the class_name is not in the dictionary (e.g., 'Сил', 'Экпарцет')
    return group_mapping.get(class_name, 'Miscellaneous')

# Step 3: Apply the function to create a new column
df['group_class'] = df['class'].apply(map_class_to_group)

# Show the updated DataFrame
print(df)

         red       date  elevation_contour  district_name  soil_name  \
0      0.217 2020-06-04               1544              2          4   
1      0.215 2020-08-03               1544              2          4   
2      0.150 2020-09-02               1544              2          4   
3      0.052 2021-04-10               1544              2          4   
4      0.056 2021-05-05               1544              2          4   
...      ...        ...                ...            ...        ...   
22923  0.192 2022-05-08                672              6          9   
22924  0.234 2022-06-07                672              6          9   
22925  0.157 2022-07-02                672              6          9   
22926  0.163 2022-08-01                672              6          9   
22927  0.133 2022-10-05                672              6          9   

           class   id group_class  
0      Картофель  986  Root Crops  
1      Картофель  986  Root Crops  
2      Картофель  986  Root

In [7]:
df['class']= label_encoder.fit_transform(df['class']) 
df['group_class']= label_encoder.fit_transform(df['group_class']) 


In [8]:
# Convert date to multiple columns (year, month, day)
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['day'] = pd.to_datetime(df['date']).dt.day
df.drop('date', axis=1, inplace=True)

In [9]:
df

,red,elevation_contour,district_name,soil_name,class,id,group_class,year,month,day
0,0.217,1544,2,4,2,986,5,2020,6,4
1,0.215,1544,2,4,2,986,5,2020,8,3
2,0.150,1544,2,4,2,986,5,2020,9,2
3,0.052,1544,2,4,2,987,5,2021,4,10
4,0.056,1544,2,4,2,987,5,2021,5,5
...,...,...,...,...,...,...,...,...,...,...
22923,0.192,672,6,9,4,752,0,2022,5,8
22924,0.234,672,6,9,4,752,0,2022,6,7
22925,0.157,672,6,9,4,752,0,2022,7,2
22926,0.163,672,6,9,4,752,0,2022,8,1


In [10]:
# Pivot table
pivot_df = df.pivot_table(index='id', columns='month', values='red', aggfunc='mean')

# Fill NaN values (assuming you want to fill with zeros, adjust if needed)
pivot_df = pivot_df.fillna(0)

# Rename columns as needed
pivot_df.columns = [f'red_{col}_month' for col in pivot_df.columns]

# Reset the index so 'id' becomes a column
pivot_df = pivot_df.reset_index()

# Assuming each 'id' has a unique 'class', get the 'class' value for each 'id' and add to the pivot dataframe
pivot_df['class'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['group_class'])
pivot_df['elevation_contour'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['elevation_contour'])
pivot_df['district_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['district_name'])
pivot_df['soil_name'] = pivot_df['id'].map(df.drop_duplicates(subset='id').set_index('id')['soil_name'])

# Reordering columns
pivot_df = pivot_df[['red_4_month', 'red_5_month', 'red_6_month', 'red_8_month', 'red_9_month', 'id', 'elevation_contour', 'district_name', 'soil_name', 'class']]


In [11]:
# Splitting the data
X = pivot_df.drop(['id', 'class'], axis=1)  # Features excluding 'id' and 'class'
y = pivot_df['class']  # Target variable

In [12]:
# imputer = KNNImputer(n_neighbors=5)
# X = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

In [13]:
class_counts = y.value_counts()

# Identify the classes with 99 or fewer samples
small_sample_classes = class_counts[class_counts <= 99].index

# Create a mask for these classes
small_sample_mask = y.isin(small_sample_classes)

# Set the values for these classes to 0
y[small_sample_mask] = 0

In [14]:
class_counts = y.value_counts()
single_sample_classes = class_counts[class_counts <= 2].index
filter_mask = ~y.isin(single_sample_classes)
X = X[filter_mask]
y = y[filter_mask]

In [15]:
y = pd.Series(y)
print('Class distribution before undersampling:\n', y.value_counts())


Class distribution before undersampling:
 class
0    647
5    204
1    144
Name: count, dtype: int64


In [16]:
rus = RandomUnderSampler(sampling_strategy='auto')
X_undersampled, y_undersampled = rus.fit_resample(X, y)

In [17]:
y_undersampled = pd.Series(y_undersampled)
print('Class distribution after undersampling:\n', y_undersampled.value_counts())

Class distribution after undersampling:
 class
0    144
1    144
5    144
Name: count, dtype: int64


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_undersampled, y_undersampled, test_size=0.2, stratify=y_undersampled)

In [19]:
# Defining the parameter grid for GradientBoostingClassifier
param_dist = {
    'n_estimators': [10, 50, 100, 200, 500],
    'learning_rate': [0.001, 0.01, 0.1, 0.5, 1],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiating GradientBoostingClassifier
gbm = GradientBoostingClassifier()

# Setting up RandomizedSearchCV
random_search = RandomizedSearchCV(
    gbm, param_distributions=param_dist, n_iter=100, scoring='accuracy', 
    cv=5, verbose=1, n_jobs=-1
)

# Fitting the RandomizedSearchCV object to the data
random_search.fit(X_train, y_train)

# Getting the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("Best Score:", random_search.best_score_)

# Evaluate on the test set
print("Test Set Score:", random_search.score(X_test, y_test))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best Parameters: {'subsample': 0.8, 'n_estimators': 200, 'max_features': None, 'max_depth': 3, 'learning_rate': 0.5}
Best Score: 0.5855072463768116
Test Set Score: 0.5172413793103449


In [20]:
print("Train Set Score:", random_search.score(X_train, y_train))

Train Set Score: 1.0
